In [58]:
import requests
import pickle
import json
import os
import googlemaps
from datetime import datetime
import sqlite3
import pandas as pd
from ast import literal_eval

gmaps = googlemaps.Client(key=api_key)

In [59]:
with open('../credentials/google_transit_api.pkl', 'rb') as hnd:
    api_key = pickle.load(hnd)

In [60]:
now = datetime.now()
date = str(now.date())
time = str(now.strftime('%H:%M'))
driving_parameters = {'origin' : "120 Benchley Pl, Bronx NY"
                     , 'destination' : "809 Washington Street"
                     , 'mode' : "driving"
                     , 'departure_time' : now}

transit_parameters = {'origin' : "120 Benchley Pl, Bronx NY"
                     , 'destination' : "809 Washington Street"
                     , 'mode' : "transit"
                     , 'departure_time' : now}
param_set_list = [driving_parameters, transit_parameters]

In [61]:
os.makedirs('data', exist_ok=True)
db_loc = 'data/transit.db'
con = sqlite3.connect(db_loc)

In [62]:
updated_params_list = []
for param_set in param_set_list:
    param_set2 = param_set.copy()
    temp_result = gmaps.directions(**param_set)
    param_set2.update({'Raw Response' : temp_result
                      , 'Date' : date
                      , 'Time' : time})
    updated_params_list.append(param_set2)
updated_params_df = pd.DataFrame(updated_params_list)
updated_params_df

,origin,destination,mode,departure_time,Raw Response,Date,Time
0,"120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-09-03 09:07:10.445214,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-09-03,09:07
1,"120 Benchley Pl, Bronx NY",809 Washington Street,transit,2021-09-03 09:07:10.445214,[{'bounds': {'northeast': {'lat': 40.874488100...,2021-09-03,09:07


In [63]:
updated_params_df = pd.DataFrame(updated_params_list)
updated_params_df['Raw Response'] = updated_params_df['Raw Response'].astype(str)
updated_params_df['departure_time'] = updated_params_df['departure_time'].astype(str)
updated_params_df

,origin,destination,mode,departure_time,Raw Response,Date,Time
0,"120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-09-03 09:07:10.445214,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-09-03,09:07
1,"120 Benchley Pl, Bronx NY",809 Washington Street,transit,2021-09-03 09:07:10.445214,[{'bounds': {'northeast': {'lat': 40.874488100...,2021-09-03,09:07


In [64]:
updated_params_df.to_sql('Raw_Responses', con, index=False, if_exists='append' )
con.commit()

/home/malcolm/main/lib/python3.6/site-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [65]:
updated_params_df['Raw Response_parsed'] = updated_params_df['Raw Response'].apply(lambda x: literal_eval(x))

In [66]:
updated_params_df['Raw Response_parsed'].iloc[0]

[{'bounds': {'northeast': {'lat': 40.8745759, 'lng': -73.8244052},
   'southwest': {'lat': 40.7391335, 'lng': -74.00886710000002}},
  'copyrights': 'Map data ©2021 Google',
  'legs': [{'distance': {'text': '18.6 mi', 'value': 29890},
    'duration': {'text': '47 mins', 'value': 2825},
    'duration_in_traffic': {'text': '45 mins', 'value': 2723},
    'end_address': '809 Washington St, New York, NY 10014, USA',
    'end_location': {'lat': 40.7391335, 'lng': -74.0080765},
    'start_address': '120 Benchley Pl, Bronx, NY 10475, USA',
    'start_location': {'lat': 40.87448810000001, 'lng': -73.82567019999999},
    'steps': [{'distance': {'text': '210 ft', 'value': 64},
      'duration': {'text': '1 min', 'value': 23},
      'end_location': {'lat': 40.87422850000001, 'lng': -73.8257921},
      'html_instructions': 'Head <b>northwest</b> toward <b>Benchley Pl</b>',
      'polyline': {'points': 'qhnxFlaraMA?A@A?A@A@A@?@A??@A@?@?@?@?@A??@?@?@?@@??@?@?@?@@??@?@@??@@??@@@@??@@?@@@?@?@?@?@?j@['},

In [3]:


# Geocoding an address
# geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')

# # Look up an address with reverse geocoding
# reverse_geocode_result = gmaps.reverse_geocode((40.872667, -73.825169))

# Request directions via public transit
driving_directions_result = gmaps.directions("120 Benchley Pl, Bronx NY",
                                     "809 Washington Street",
                                     mode="driving", 
                                     # mode='transit',
                                     departure_time=now)

transit_directions_result = gmaps.directions("120 Benchley Pl, Bronx NY",
                                     "809 Washington Street",
#                                      mode="driving", 
                                     mode='transit',
                                     departure_time=now)

In [8]:
date = str(now.date())
transit_save_loc = f'data/transit/{date}/'
driving_save_loc = f'data/driving/{date}/'
os.makedirs(transit_save_loc, exist_ok=True)
os.makedirs(driving_save_loc, exist_ok=True)
time = now.strftime('%H:%M')
time

'00:27'

In [11]:
with open(transit_save_loc + time + '.json', 'w') as hnd:
    json.dump(transit_directions_result, hnd)

In [10]:
with open(driving_save_loc + time + '.json', 'w') as hnd:
    json.dump(driving_directions_result, hnd)

In [4]:
print(len(directions_result))
directions_result[0].keys()

1


dict_keys(['bounds', 'copyrights', 'legs', 'overview_polyline', 'summary', 'warnings', 'waypoint_order'])

In [5]:
directions_result[0]['summary']

'I-678 S'

In [6]:
legs = directions_result[0]['legs']
print(legs[0].keys())
legs

dict_keys(['distance', 'duration', 'duration_in_traffic', 'end_address', 'end_location', 'start_address', 'start_location', 'steps', 'traffic_speed_entry', 'via_waypoint'])


[{'distance': {'text': '20.8 mi', 'value': 33530},
  'duration': {'text': '45 mins', 'value': 2719},
  'duration_in_traffic': {'text': '35 mins', 'value': 2077},
  'end_address': '508 LaGuardia Pl, New York, NY 10012, USA',
  'end_location': {'lat': 40.7277026, 'lng': -73.9992836},
  'start_address': '120 Benchley Pl, Bronx, NY 10475, USA',
  'start_location': {'lat': 40.87448810000001, 'lng': -73.82567019999999},
  'steps': [{'distance': {'text': '210 ft', 'value': 64},
    'duration': {'text': '1 min', 'value': 23},
    'end_location': {'lat': 40.87422850000001, 'lng': -73.8257921},
    'html_instructions': 'Head <b>northwest</b> toward <b>Benchley Pl</b>',
    'polyline': {'points': 'qhnxFlaraMA?A@A?A@A@A@?@A??@A@?@?@?@?@A??@?@?@?@@??@?@?@?@@??@?@@??@@??@@@@??@@?@@@?@?@?@?@?j@['},
    'start_location': {'lat': 40.87448810000001, 'lng': -73.82567019999999},
    'travel_mode': 'DRIVING'},
   {'distance': {'text': '187 ft', 'value': 57},
    'duration': {'text': '1 min', 'value': 56},


In [7]:
legs[0]['duration']

{'text': '45 mins', 'value': 2719}

In [8]:
len(legs[0]['steps'])
legs[0]['steps']

[{'distance': {'text': '210 ft', 'value': 64},
  'duration': {'text': '1 min', 'value': 23},
  'end_location': {'lat': 40.87422850000001, 'lng': -73.8257921},
  'html_instructions': 'Head <b>northwest</b> toward <b>Benchley Pl</b>',
  'polyline': {'points': 'qhnxFlaraMA?A@A?A@A@A@?@A??@A@?@?@?@?@A??@?@?@?@@??@?@?@?@@??@?@@??@@??@@@@??@@?@@@?@?@?@?@?j@['},
  'start_location': {'lat': 40.87448810000001, 'lng': -73.82567019999999},
  'travel_mode': 'DRIVING'},
 {'distance': {'text': '187 ft', 'value': 57},
  'duration': {'text': '1 min', 'value': 56},
  'end_location': {'lat': 40.873769, 'lng': -73.82609219999999},
  'html_instructions': 'Turn <b>right</b> onto <b>Benchley Pl</b>',
  'maneuver': 'turn-right',
  'polyline': {'points': '}fnxFdbraMLJLDJFNJXNHD'},
  'start_location': {'lat': 40.87422850000001, 'lng': -73.8257921},
  'travel_mode': 'DRIVING'},
 {'distance': {'text': '463 ft', 'value': 141},
  'duration': {'text': '1 min', 'value': 49},
  'end_location': {'lat': 40.873684999999

In [9]:
[x['travel_mode'] for x in legs[0]['steps']]

['DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING',
 'DRIVING']

In [10]:
transit_steps = [x for x in legs[0]['steps'] if x['travel_mode'] == 'TRANSIT']
print(len(transit_steps))
transit_steps

0


[]

In [11]:
transit_steps[0].keys()

IndexError: list index out of range

In [71]:
transit_steps[1]['transit_details']['line']['short_name']

'D'

In [72]:
[x['transit_details']['line']['short_name'] for x in transit_steps]

['Bx12', 'D']

In [73]:
[x['transit_details']['departure_time'] for x in transit_steps]

[{'text': '12:18am', 'time_zone': 'America/New_York', 'value': 1626236322},
 {'text': '1:11am', 'time_zone': 'America/New_York', 'value': 1626239490}]

In [74]:
[x['transit_details']['arrival_time'] for x in transit_steps]

[{'text': '12:43am', 'time_zone': 'America/New_York', 'value': 1626237835},
 {'text': '1:47am', 'time_zone': 'America/New_York', 'value': 1626241674}]

In [75]:
[x['transit_details']['num_stops'] for x in transit_steps]

[17, 14]

In [76]:
[x['duration'] for x in transit_steps]

[{'text': '25 mins', 'value': 1513}, {'text': '36 mins', 'value': 2184}]

In [ ]:
# 